O objetivo deste trabalho é comparar diversos métodos de classificação para a base de dados de qualidade de vinhos disponível em https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv.

Vocês devem encontrar um bom modelo preditivo, variando:
* o número e conjunto de features (atributos) utilizados
* o método utilizado
* a configuração do algoritmo correspondente (e.g.: número k para nearest neighbors, profundidade para árvore de decisão)

Vocês devem listar algumas métricas de qualidade, tais como: precision, recall, accuracy e f1_score, e utilizar accuracy como base para a avaliação final, considerando a accuracy média de 10 iterações para cada configuração.

Para assegurar que eu obterei os mesmos resultados de vocês, vocês devem estabelecer a semente para a geração dos números aleatórios (utilizados para separar os conjuntos de treinamento e teste, por exemplo), utilizando os seguintes comandos no início do seu código (podem utilizar uma outra semente):
```
import random
random.seed(1001001)
```

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

%matplotlib inline

In [3]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv' ,sep=';')

df.head()

URLError: <urlopen error [Errno -2] Name or service not known>

In [ ]:
df.quality.describe()

In [ ]:
df.quality.unique()

In [ ]:
X = df.iloc[:, :-1].values
Y = df.iloc[:,-1].values

import random
random.seed(1001001)

from sklearn.preprocessing import normalize

X = normalize(X)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, Y_train)

In [ ]:
y_pred = regressor.predict(X_test)

y_round = []
for y in y_pred:
    y_round.append(int(round(y)))

In [ ]:
np.mean(Y_test == y_round)

### SVM

In [ ]:
from sklearn.svm import SVC

svm = SVC(C = 6, kernel = 'linear')
svm.fit(X_train, Y_train)

In [ ]:
svm.score(X_test, Y_test)

### K-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 50).fit(X_train, Y_train)

In [ ]:
knn.score(X_test, Y_test)

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=60)
clf.fit(X_train, Y_train)   

In [ ]:
clf.score(X_test, Y_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(X_train, Y_train)

In [ ]:
dt.score(X_test, Y_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(X_train, Y_train)
gnb_predictions = gnb.predict(X_test)

gnb.score(X_test, Y_test)

### Neural Network

In [ ]:
D = X.shape[1] # input dim
M = 11 # hidden size
K = 6 # number classes

In [ ]:
T = np.zeros((Y.size, K))
for i in range(Y.size):
    T[i, Y[i]-3] = 1

T_train = np.zeros((Y_train.size, K))
for i in range(Y_train.size):
    T_train[i, Y_train[i]-3] = 1
    
T_test = np.zeros((Y_test.size, K))
for i in range(Y_test.size):
    T_test[i, Y_test[i]-3] = 1

In [ ]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.1))

W1 = init_weights([D, M])
b1 = init_weights([M])

W2 = init_weights([M, M])
b2 = init_weights([M])

W3 = init_weights([M, M])
b3 = init_weights([M])

Wf = init_weights([M, K])
bf = init_weights([K])

In [ ]:
tfX = tf.placeholder(tf.float32, [None, D])
tfY = tf.placeholder(tf.float32, [None, K])

def forward():
    Z = tf.nn.sigmoid(tf.matmul(tfX, W1) + b1)
    Z2 = tf.nn.sigmoid(tf.matmul(Z, W2) + b2)
    Z3 = tf.nn.sigmoid(tf.matmul(Z2, W3) + b3)
    
    return tf.matmul(Z3, Wf) + bf

py_x = forward()

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tfY, logits=py_x))

In [ ]:
train_op = tf.train.AdamOptimizer().minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

from tqdm import tqdm
pbar= tqdm(range(15000))

for i in pbar:
    sess.run(train_op, feed_dict={tfX: X_train, tfY: T_train})
    pred_train = sess.run(predict_op, feed_dict={tfX: X_train, tfY: T_train})
    pred_test = sess.run(predict_op, feed_dict={tfX: X_test, tfY: T_test})
    
    if i%1000 == 0:
        pbar.set_description("acc train: {}; acc test: {}".format(np.mean(Y_train-3 == pred_train), 
                                                                  np.mean(Y_test-3 == pred_test)))

In [ ]:
tf.Print(W1, [W1])